# Project Physics

In this text we will look, numerically, at the stability of a ship. For a ship to be able to sail across the ocean, it needs to have a certain amount of stability. This stability prevents it from capsizing from the forces that are bound to be affecting the ship on its tour. We will look at how different initial conditions, as well as other factors, such as waves, affects the stability of a ship.

To make the calculations accessible, we need to make simplifications of the ship. Firstly, we will make the problem 2-dimensional by only looking at a slice in the middle of the ship. This simplification does not affect the final result considerably, as it is safe to assume that the forces are close to constant throughout the ship, and our interest is in whether the ship will capsize.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

First of, we need to define our ship. Our ship will be a half-circle with uniform mass (see figure 1**). We need several constants that we will use in the calculations:

In [3]:
R = 10 # Radius of the ship
H = 4 * R / (3 * np.pi) # The distance from the midpoint of the diameter to the center of mass
g = 9.81 # The gravitational acceleration
sigma0 = 1000 # The density of the water
sigma = 500 # The density of the ship

To do some calculations, we first need to identify the forces that act on the ship. We will start by assuming the center of mass stays in place, and the only movement will be a rotation of the ship. As a start, the forces analyzed will be restricted to the updraft of the water. This updraft will, by symmetry, be normal to the surface of the water. With the use of geometry one can find that the area subdued underwater is
$$A = \frac{1}{2}R^2(\gamma-\sin(\gamma))$$
Furthermore, we define the angle $\theta$ as the angle of rotation around the center of mass of the boat. This angle is defined positive counter-clockwise. With further geometry, it can be shown that the angle $\gamma$ can be expressed by $\theta$:
$$\gamma=2 \arccos{\left(\cos(\beta/2)-\frac{4}{3\pi}(1-\cos(\theta)\right)$$
where $\beta$ is the initial $\gamma$ when the ship is in equilibrium.

Now, the total updraft from the water is
$$F_B=A\sigma_0g$$
As a rotation of the ship will displace the center of mass from the center of the updraft force. This gives a torque that can be expressed as:
$$\tau_B = -F_Bh\sin(\theta)$$
It can also be shown that the moment of inertia of the ship around the center of mass is:
$$I_C = \frac{1}{2}mR^2-mH^2$$
Where H is the distance from the midpoint M (see figure 1) to the center of mass C.

We will now use Eulers method to simulate numerically the movement of the ship:

In [5]:
A0 = 1 / 2 * R ** 2 * np.pi * sigma / sigma0 # Area subdued underwater at equilibrium
m = A0 * sigma0 # Mass of the water that is displaced
I_M = 1 / 2 * m * R ** 2 # Moment of inertia of the midpoint M
I_C = I_M - m * H ** 2 # Moment of inertia of the center of mass
omega_0 = (m * g * H / I_C) ** 0.5 # ??

F_G = -m * g # ??

def F_B(A): # The updraft force
    return A * sigma0 * g

def tau_B(theta, A): #The torque from the updraft
    return -F_B(A) * H * np.sin(theta)

In [6]:
def f(x, y):  # Function that defines the derivative of theta and w
    theta, w = y
    tau = tau_B(theta, A0)
    return np.array([w, tau / I_C])